In [1]:
#!pip install minsearch

In [2]:
import minsearch, json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
#docs_raw

In [5]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=['question','section','text'],
    keyword_fields=['course']
)

In [8]:
index.fit(documents)

In [9]:
q = 'course already started, can i still enroll'

In [10]:
# boost = {'question':3.0, 'text':0.5}

# results = index.search(
#     query=q,
#     boost_dict=boost,
#     num_results = 10
# )

In [11]:
from google import genai
import os
os.environ["GOOGLE_API_KEY"] = "" #"API KEY HERE"

In [12]:
client = genai.Client()

response = client.models.generate_content(
            model="gemini-2.0-flash", 
            contents="Explain how AI works in a few words?"
        )
print(response.text)

AI learns patterns from data to make predictions or decisions.



In [13]:
def search(query):
    boost = {'question':3.0, 'text':0.5}
    
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results = 10
        )
    return results
    

In [14]:
def build_prompt(query, search_results):
    prompt_template = """
        You are a course teaching assistant. 
        Answer the QUESTION based on the CONTEXT from the FAQ document
        Use only the facts from the CONTEXT while answering the QUESTION
        If the CONTEXT does not contain the answer, output NONE
        QUESTION: {question}
        CONTEXT: {context}
        
        """.strip()
    context =""
    for doc in search_results:
        context = context + f"section: {doc['section']}\n question: {doc['question']}\n answer: {doc['text']}\n\n " 
    #print(context)
    prompt = prompt_template.format(question = query, context =context).strip()
    #print(query)
    return prompt

In [15]:
def llm(prompt):
    #lets do it in gemini
    response = client.models.generate_content(
                model="gemini-2.0-flash", 
                contents= prompt 
            )
    return response.text

In [16]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return(answer)

In [17]:
rag('course already started, can i still enroll?')

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [18]:
rag("how do I run kafka")

'```text\nIn the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```'

In [19]:
rag('How do i enroll in the course')

"Register before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel."

In [21]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [22]:
from elasticsearch import Elasticsearch

In [24]:
es_client = Elasticsearch('http://localhost:9200')

In [25]:
es_client.info()

ObjectApiResponse({'name': '0cc6361a58b0', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'KjTukG7gQjiVjPTRytLvSA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [26]:
es_client.info()

ObjectApiResponse({'name': '0cc6361a58b0', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'KjTukG7gQjiVjPTRytLvSA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [28]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index = index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [29]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [30]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
for doc in documents:
    es_client.index(index = index_name, document= doc)

In [33]:
query = "how can i run kafka"

In [34]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [35]:
response = es_client.search(index=index_name, body=search_query)

In [36]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'SBF4g5cBWB-XZKh0JAl6',
  '_score': 29.75321,
  '_source': {'text': 'In Confluent Cloud:\nEnvironment → default (or whatever you named your environment as) → The right navigation bar →  “Stream Governance API” →  The URL under “Endpoint”\nAnd create credentials from Credentials section below it',
   'section': 'Module 6: streaming with kafka',
   'question': 'Confluent Kafka: Where can I find schema registry URL?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'RRF4g5cBWB-XZKh0JAlx',
  '_score': 26.374605,
  '_source': {'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
   'section': 'Module 6: streaming with kafka',
   'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'XhF4g5cBWB-XZKh0JAnF',
  '_score': 23

In [37]:
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [38]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index = index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs



In [39]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return(answer)

In [40]:
rag('how can i run kafka')

'```text\nTo run producer/consumer/kstreams in the terminal, navigate to the project directory and run: java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```'

In [41]:
rag('how can i enroll in course')

'NONE\n'

In [42]:
rag('Course already started can i still enroll')

"Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."